In [3]:
!pip install torch torchvision pillow --quiet


In [8]:
import torch  # PyTorch for deep learning operations
import torchvision.models as models  # Pretrained models like ResNet
import torchvision.transforms as transforms  # Image transformations
from PIL import Image, UnidentifiedImageError  # Image processing and error handling
import requests  # HTTP requests to download images and labels
from requests.exceptions import RequestException  # Handle connection errors

# 1. Load Pretrained ResNet-18 Model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)  # Load ResNet-18 pretrained on ImageNet
model.eval()  # Set the model to evaluation mode (faster inference, no gradient calculations)

# 2. Image URLs for Batch Processing (Cats and Dogs)
# A list of valid image URLs (from Unsplash) to be classified
image_urls = [
    "https://images.unsplash.com/photo-1574158622682-e40e69881006",  # Dog
    "https://images.unsplash.com/photo-1517849845537-4d257902454a",  # Cat
    "https://images.unsplash.com/photo-1555685812-4b943f1cb0eb",      # Dog
    "https://images.unsplash.com/photo-1518791841217-8f162f1e1131"   # Cat
]

# 3. Transformations for ResNet
# Resize, convert to tensor, and normalize images to fit ResNet's input format
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224 (ResNet input size)
    transforms.ToTensor(),  # Convert image to tensor (for PyTorch model)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and std
])

# 4. Download and Transform Each Image (Skip Invalid and Unreachable URLs)
batch_images = []  # List to store transformed images
for url in image_urls:
    try:
        # Send a GET request to download the image
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an error if the request returns a 404 or 500 error

        # Open the image from the response stream
        image = Image.open(response.raw)

        # Apply transformations and add the image to the batch
        batch_images.append(transform(image))

    except (UnidentifiedImageError, RequestException) as e:
        # If the image can't be opened or downloaded, print an error and skip the URL
        print(f"Skipping URL: {url} - {str(e)}")

# Ensure at least one valid image is in the batch, or raise an error
if len(batch_images) == 0:
    raise ValueError("No valid images could be processed. Please check URLs.")

# Stack all valid images into a single batch tensor for model input
batch_input = torch.stack(batch_images)

# 5. Perform Inference on Batch
output = model(batch_input)  # Pass the batch through the model to get predictions

# 6. Get Top Predictions for Each Image
predicted_classes = torch.argmax(output, dim=1)  # Get the class with the highest score for each image

# 7. Download ImageNet Class Labels
LABELS_URL = "https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json"
labels = requests.get(LABELS_URL).json()  # Download class index to map predictions to human-readable labels

# 8. Map Class Index to Human-Readable Labels
for i, class_idx in enumerate(predicted_classes):
    class_name = labels[str(class_idx.item())][1]  # Get label for predicted class index
    print(f"Image {i+1}: Predicted Class - {class_name}")  # Print the predicted class name for each image


Image 1: Predicted Class - tabby
Image 2: Predicted Class - pug
Image 3: Predicted Class - Egyptian_cat
Image 4: Predicted Class - tabby
